In [2]:
import matplotlib.pyplot as plt
from scipy import sparse
import numpy as np
import pickle

In [4]:
#load everything
num_ing = 8023
recipe_decoder = pickle.load( open('data-cleaned/recipe_decoder.pkl', 'rb') )
ingr_decoder = pickle.load( open('data-cleaned/ingredient_decoder.pkl', 'rb') )
tag_decoder = pickle.load( open('data-cleaned/tag_decoder.pkl', 'rb') )
X = sparse.load_npz("data-cleaned/recipes.npz")
Xhat = sparse.load_npz("data-cleaned/recipes_tfidf.npz")
U = sparse.load_npz("data-cleaned/user_train.npz")
Uhat = sparse.load_npz("data-cleaned/user_train_tfidf.npz")

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [3]:
def recipe_info(i):
    #extract info
    calories = X[i, -1]
    idx = X[i].nonzero()[1]
    ingredients = [ingr_decoder[j] for j in idx[idx < num_ing]]
    tags = [tag_decoder[j] for j in idx[idx > num_ing][:-1] - num_ing]
    name = recipe_decoder[i]
    
    #print
    print(f"Recipe: {name}")
    print(f"Calorie Level: {calories}")
    print(f"Ingredients: {ingredients}")
    print(f"Tags: {tags}")

In [4]:
def recipe_comp(i, j):
    #extract info
    cali = X[i, -1]
    idxi = X[i].nonzero()[1]
    ingi = [ingr_decoder[k] for k in idxi[idxi < num_ing]]
    tagsi = [tag_decoder[k] for k in idxi[idxi > num_ing][:-1] - num_ing]
    namei = recipe_decoder[i]
    
    calj = X[j, -1]
    idxj = X[j].nonzero()[1]
    ingj = [ingr_decoder[k] for k in idxj[idxj < num_ing]]
    tagsj = [tag_decoder[k] for k in idxj[idxj > num_ing][:-1] - num_ing]
    namej = recipe_decoder[j]
    
    #print
    print(f"Recipe: {namei}, {namej}")
    print(f"Difference Calorie Level: {cali-calj}")
    print(f"Shared Ingredients: {intersection(ingi, ingj)}")
    print(f"Tags: {intersection(tagsi, tagsj)}")

In [1]:
def recipe_score(i, j):
    #set things up 
    score = np.zeros_like(i)
    
    #iterate through all values
    for num, (k, l) in enumerate(zip(i, j)):
        xk = X[k, :-1].nonzero()[1]
        xl = X[l, :-1].nonzero()[1]
        #add all identical tags and if calories are the same (don't double count calories)
        score[num] = len(intersection(xk, xl)) + int(X[k,-1]==X[l,-1])
        
    return score